In [ ]:
# Data Handling
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import re

# Visualization
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from palettable.colorbrewer.qualitative import Pastel1_7


# Data pre-processing - Class Balancing and Scaling
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Feature Dimension Engineering
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import statsmodels.api as sm
from xgboost import XGBClassifier

# Model Tuning 
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [ ]:
# Calculate Metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,precision_recall_curve 
from sklearn import metrics
from scipy.stats import norm

# Set the session options
%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = None
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Read data into Dataframe
cust_data = pd.read_csv('telecom_churn_data.csv')


In [ ]:
cust_data.head()

In [ ]:
# Check Data Dimensions, data types and the NaN distribution
print(cust_data.shape)
print('\n ---- Null Values ---- ')
print(cust_data.isnull().sum(axis=0).sort_values(ascending=False))

# Identify High Value Customers and assign Churn label as applicable

In [ ]:
# Sanity check for average and total data recharge features
cols=cust_data.columns
av_reg=re.compile(r'av_rech_amt_data_*')
tot_reg=re.compile(r'total_rech_data_*')
data_rech_cols=list(filter(av_reg.match,cols))+list(filter(tot_reg.match,cols))
cust_data[data_rech_cols].isnull().sum(axis=0).sort_values(ascending=False)

In [ ]:
# As there are lot of NaNs, dropping rows is not correct method. 
#Imput NaNs with 0s and then caluclate the total recharge value for 'Good' months.
cust_data[data_rech_cols]=cust_data[data_rech_cols].fillna(0)
cust_data['total_data_rech_amt_6']=cust_data['av_rech_amt_data_6']*cust_data['total_rech_data_6']
cust_data['total_data_rech_amt_7']=cust_data['av_rech_amt_data_7']*cust_data['total_rech_data_7']

# Identify the high value customers. Any customers who have recharged more than or equal to 70 perctile of the grand recharge average of good months ( month 6 and 7)
cust_data['avg_good_month_rech_amt']=(cust_data['total_rech_amt_6']+cust_data['total_rech_amt_7']+cust_data['total_data_rech_amt_6']+cust_data['total_data_rech_amt_7'])/2
cust_data['high_value_cust']=np.where(cust_data['avg_good_month_rech_amt']>np.percentile(cust_data['avg_good_month_rech_amt'],70),'Y','N')
print('70th percentile of the grand average of good months:  ' + str(np.percentile(cust_data['avg_good_month_rech_amt'],70))+'\n')

# Count of High Value Customers
print('High value Customers: ',"\n", cust_data.groupby(['high_value_cust'])['mobile_number'].count(),"\n")

# Copy  the High Value customers
hv_cust_data=cust_data[cust_data['high_value_cust']=='Y']

In [ ]:
# Count of churn Customers
hv_cust_data['churn']=np.where(((hv_cust_data['total_ic_mou_9']==0) & 
                             (hv_cust_data['total_og_mou_9']==0) &
                             (hv_cust_data['vol_2g_mb_9']==0) & 
                             (hv_cust_data['vol_3g_mb_9']==0)),1,0)


print('Churn Customers in High Value Customers',"\n", hv_cust_data[hv_cust_data['high_value_cust']=='Y'].groupby(['churn'])['mobile_number'].count())

# Drop the columns related to churn month
cols=hv_cust_data.columns
churn_reg=re.compile(r'.*_9')
churn_cols=list(filter(churn_reg.match,cols))
churn_cols=churn_cols+['sep_vbc_3g']
hv_cust_data.drop(columns=churn_cols,axis=1,inplace=True)

# Apply standard Data Cleaning techniques, Treat Outliers , Derive Features as applicable

In [ ]:
# Sanity Check- Display shape, Number of NaNs in columns and datatypes
print(hv_cust_data.shape)
hv_cust_data.isnull().sum(axis=0).sort_values(ascending=False)
print(hv_cust_data.dtypes)


In [ ]:
# Convert date time fields to datatime objects
hv_cust_data['last_date_of_month_6'] = pd.to_datetime(hv_cust_data['last_date_of_month_6'])
hv_cust_data['last_date_of_month_7'] = pd.to_datetime(hv_cust_data['last_date_of_month_7'])
hv_cust_data['last_date_of_month_8'] = pd.to_datetime(hv_cust_data['last_date_of_month_8'])
hv_cust_data['date_of_last_rech_6'] = pd.to_datetime(hv_cust_data['date_of_last_rech_6'])
hv_cust_data['date_of_last_rech_7'] = pd.to_datetime(hv_cust_data['date_of_last_rech_7'])
hv_cust_data['date_of_last_rech_8'] = pd.to_datetime(hv_cust_data['date_of_last_rech_8'])
hv_cust_data['date_of_last_rech_data_6'] = pd.to_datetime(hv_cust_data['date_of_last_rech_data_6'])
hv_cust_data['date_of_last_rech_data_7'] = pd.to_datetime(hv_cust_data['date_of_last_rech_data_7'])
hv_cust_data['date_of_last_rech_data_8'] = pd.to_datetime(hv_cust_data['date_of_last_rech_data_8'])

# Derive new features using the dates
hv_cust_data['rech_before_lastday_6'] = (hv_cust_data['last_date_of_month_6']-hv_cust_data['date_of_last_rech_6']).apply(lambda x: x.days)
hv_cust_data['rech_before_lastday_7'] = (hv_cust_data['last_date_of_month_7']-hv_cust_data['date_of_last_rech_7']).apply(lambda x: x.days)
hv_cust_data['rech_before_lastday_8'] = (hv_cust_data['last_date_of_month_8']-hv_cust_data['date_of_last_rech_8']).apply(lambda x: x.days)

hv_cust_data['rechdata_before_lastday_6'] = (hv_cust_data['last_date_of_month_6']-hv_cust_data['date_of_last_rech_data_6']).apply(lambda x: x.days)
hv_cust_data['rechdata_before_lastday_7'] = (hv_cust_data['last_date_of_month_7']-hv_cust_data['date_of_last_rech_data_7']).apply(lambda x: x.days)
hv_cust_data['rechdata_before_lastday_8'] = (hv_cust_data['last_date_of_month_8']-hv_cust_data['date_of_last_rech_data_8']).apply(lambda x: x.days)

#Rename columns for easy identification
hv_cust_data.columns=hv_cust_data.columns.str.replace('_8+','_active')

# Take a copy of the dataset for EDA purpose
hv_cust_data_copy=hv_cust_data.copy()

#Do a visual check of the data set
hv_cust_data.head()

In [ ]:
# Feature: arpu
cols=['arpu_6','arpu_7','arpu_active']
hv_cust_data[cols].isnull().sum(axis=0).sort_values(ascending=False)

In [ ]:
hv_cust_data['arpu_good']=(hv_cust_data['arpu_6']+hv_cust_data['arpu_7'])/2
hv_cust_data[['arpu_good','arpu_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])


In [ ]:

hv_cust_data[hv_cust_data['arpu_active']>np.percentile(hv_cust_data['arpu_active'],99.95)][['arpu_good','arpu_active','churn']]

In [ ]:
hv_cust_data=hv_cust_data[~(hv_cust_data['arpu_active']>np.percentile(hv_cust_data['arpu_active'],99.95))]
hv_cust_data.drop(columns=['arpu_6','arpu_7'],axis=1,inplace=True)

In [ ]:
# Feature : arpu_2g and arpu_3g
hv_cust_data[['arpu_2g_6','arpu_2g_7','arpu_2g_active','arpu_3g_6','arpu_3g_7','arpu_3g_active']].isnull().sum(axis=0).sort_values(ascending=False)

In [ ]:
hv_cust_data[['arpu_2g_6','arpu_2g_7','arpu_2g_active',]].describe(percentiles=[0.75,.995,.998,.999,1.00])


In [ ]:
hv_cust_data.drop(columns=['arpu_2g_6','arpu_2g_7','arpu_2g_active','arpu_3g_6','arpu_3g_7','arpu_3g_active'],axis=1,inplace=True)

In [ ]:
# Feature: aon
print(hv_cust_data[['aon']].isnull().sum(axis=0).sort_values(ascending=False))
print(hv_cust_data[['aon']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
# Feature : vbc
print(hv_cust_data[['aug_vbc_3g','jun_vbc_3g','jul_vbc_3g']].isnull().sum(axis=0).sort_values(ascending=False))
print(hv_cust_data[['aug_vbc_3g','jun_vbc_3g','jul_vbc_3g']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
hv_cust_data['vbc_3g_good']=(hv_cust_data['jun_vbc_3g']+hv_cust_data['jul_vbc_3g'])/2
hv_cust_data['vbc_3g_active']=hv_cust_data['aug_vbc_3g']
print(hv_cust_data[['vbc_3g_good','vbc_3g_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
hv_cust_data[hv_cust_data['vbc_3g_active']>np.percentile(hv_cust_data['vbc_3g_active'],99.97)][['vbc_3g_active','vbc_3g_good','churn']]

In [ ]:
hv_cust_data=hv_cust_data[~(hv_cust_data['vbc_3g_active']>np.percentile(hv_cust_data['vbc_3g_active'],99.97))]
hv_cust_data.drop(columns=['jun_vbc_3g','jul_vbc_3g','aug_vbc_3g'],axis=1,inplace=True)

In [ ]:
# count_reach_2g and 3g
cols=list(hv_cust_data.filter(regex='count_rech_').columns)
print(hv_cust_data[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
for i in cols:
    hv_cust_data[i]=hv_cust_data[i].fillna(0)

In [ ]:
hv_cust_data['count_rech_2g_good']=(hv_cust_data['count_rech_2g_6']+hv_cust_data['count_rech_2g_7'])/2
hv_cust_data['count_rech_3g_good']=(hv_cust_data['count_rech_3g_6']+hv_cust_data['count_rech_3g_7'])/2
print(hv_cust_data[['count_rech_2g_good','count_rech_2g_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))
print(hv_cust_data[['count_rech_3g_good','count_rech_3g_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))


In [ ]:
hv_cust_data[hv_cust_data['count_rech_2g_active']>np.percentile(hv_cust_data['count_rech_2g_active'],99.95)][['count_rech_2g_good','count_rech_2g_active','churn']]

In [ ]:
hv_cust_data=hv_cust_data[~(hv_cust_data['count_rech_2g_active']>np.percentile(hv_cust_data['count_rech_2g_active'],99.95))]
print(hv_cust_data[['count_rech_3g_good','count_rech_3g_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))
hv_cust_data[hv_cust_data['count_rech_3g_active']>np.percentile(hv_cust_data['count_rech_3g_active'],99.95)][['count_rech_3g_good','count_rech_3g_active','churn']]

In [ ]:
hv_cust_data=hv_cust_data[~(hv_cust_data['count_rech_3g_active']>np.percentile(hv_cust_data['count_rech_3g_active'],99.95))]
hv_cust_data.drop(columns=['count_rech_2g_6','count_rech_2g_7','count_rech_3g_6','count_rech_3g_7'],axis=1,inplace=True)

In [ ]:
#fb_users
cols=list(hv_cust_data.filter(regex='fb_user').columns)
print(hv_cust_data[cols].isnull().sum(axis=0).sort_values(ascending=False))
print(hv_cust_data[cols].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
# Ignore fb_user_6 and impute NaNs of fb_user_7 and fb_user_active as 2. Then use one hot encoding for fb_user_7 and fb_user active
hv_cust_data['fb_user_7']=hv_cust_data['fb_user_7'].fillna(2)
hv_cust_data['fb_user_active']=hv_cust_data['fb_user_active'].fillna(2)
fb_status_good = pd.get_dummies(hv_cust_data['fb_user_7'], drop_first = True)
fb_status_good.rename({1.0: 'fb_user_good', 2.0: 'fb_not_known_good'}, axis='columns',inplace=True)
fb_status_active = pd.get_dummies(hv_cust_data['fb_user_active'], drop_first = True)
fb_status_active.rename({1.0: 'fb_user_active', 2.0: 'fb_not_known_active'}, axis='columns',inplace=True)
hv_cust_data = pd.concat([hv_cust_data,fb_status_active,fb_status_good], axis = 1)

In [ ]:
hv_cust_data.drop(columns=cols,axis=1,inplace=True)

In [ ]:
cols=list(hv_cust_data.filter(regex='vol_').columns)
print(hv_cust_data[cols].isnull().sum(axis=0).sort_values(ascending=False))
print(hv_cust_data[cols].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
hv_cust_data['vol_2g_mb_good']=(hv_cust_data['vol_2g_mb_6']+hv_cust_data['vol_2g_mb_7'])/2
hv_cust_data['vol_3g_mb_good']=(hv_cust_data['vol_3g_mb_6']+hv_cust_data['vol_3g_mb_7'])/2
print(hv_cust_data[['vol_2g_mb_good','vol_2g_mb_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
hv_cust_data[hv_cust_data['vol_2g_mb_active']>np.percentile(hv_cust_data['vol_2g_mb_active'],99.95)][['vol_2g_mb_active','vol_2g_mb_good','churn']]

In [ ]:
hv_cust_data=hv_cust_data[~(hv_cust_data['vol_2g_mb_active']>np.percentile(hv_cust_data['vol_2g_mb_active'],99.95))]
print(hv_cust_data[['vol_3g_mb_good','vol_3g_mb_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
hv_cust_data[hv_cust_data['vol_3g_mb_active']>np.percentile(hv_cust_data['vol_3g_mb_active'],99.98)][['vol_3g_mb_active','vol_3g_mb_good','churn']]

In [ ]:
hv_cust_data=hv_cust_data[~(hv_cust_data['vol_3g_mb_active']>np.percentile(hv_cust_data['vol_3g_mb_active'],99.98))]
hv_cust_data.drop(columns=['vol_2g_mb_6','vol_2g_mb_7','vol_3g_mb_6','vol_3g_mb_7'],axis=1,inplace=True)

# Feature: monthly 2g and 3g
cols=list(hv_cust_data.filter(regex='monthly_').columns)
print(hv_cust_data[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_cust_data['monthly_2g_good']=(hv_cust_data['monthly_2g_6']+hv_cust_data['monthly_2g_7'])/2
hv_cust_data['monthly_3g_good']=(hv_cust_data['monthly_3g_6']+hv_cust_data['monthly_3g_7'])/2
hv_cust_data[['monthly_3g_good','monthly_3g_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_cust_data[hv_cust_data['monthly_3g_good']>np.percentile(hv_cust_data['monthly_3g_good'],99.96)][['monthly_3g_good','monthly_3g_active','churn']]


In [ ]:
hv_cust_data=hv_cust_data[~(hv_cust_data['monthly_3g_good']>np.percentile(hv_cust_data['monthly_3g_good'],99.96))]
hv_cust_data.drop(columns=['monthly_2g_6','monthly_2g_7','monthly_3g_6','monthly_3g_7'],axis=1,inplace=True)

In [ ]:
# Feature: Local incoming calls
cols=list(hv_cust_data.filter(regex='loc_ic').columns)
print(hv_cust_data[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_cust_data['loc_ic_mou_good']=(hv_cust_data['loc_ic_mou_6']+hv_cust_data['loc_ic_mou_7'])/2
hv_cust_data['loc_ic_mou_good'].isnull().sum(axis=0)

In [ ]:
print(hv_cust_data[~(hv_cust_data['loc_ic_mou_good'].isnull())].groupby(['churn'])['mobile_number'].count())
print(hv_cust_data[hv_cust_data['loc_ic_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_cust_data['loc_ic_null_good']=np.where((hv_cust_data['loc_ic_mou_good'].isnull()),1,0) 
hv_cust_data['loc_ic_null_active']=np.where((hv_cust_data['loc_ic_mou_active'].isnull()),1,0) 
print('Good Month Comparision \n')
print(hv_cust_data[hv_cust_data['loc_ic_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print(hv_cust_data[hv_cust_data['loc_ic_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('Active Month Comparision \n')
print(hv_cust_data[hv_cust_data['loc_ic_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())
print(hv_cust_data[hv_cust_data['loc_ic_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_cust_data['loc_ic_mou_active']=hv_cust_data[['loc_ic_mou_active']].fillna(0)
hv_cust_data['loc_ic_mou_good']=hv_cust_data[['loc_ic_mou_good']].fillna(0)
print(hv_cust_data[['loc_ic_mou_active','loc_ic_mou_good']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
hv_cust_data.drop(columns=['loc_ic_mou_6','loc_ic_mou_7','loc_ic_t2f_mou_6','loc_ic_t2f_mou_7','loc_ic_t2f_mou_active',
                          'loc_ic_t2m_mou_6','loc_ic_t2m_mou_7','loc_ic_t2m_mou_active','loc_ic_t2t_mou_6',
                          'loc_ic_t2t_mou_7','loc_ic_t2t_mou_active'],axis=1,inplace=True)

In [ ]:
# Feature: local outgoing calls
cols=list(hv_cust_data.filter(regex='loc_og').columns)
print(hv_cust_data[cols].isnull().sum(axis=0).sort_values(ascending=False))


In [ ]:
hv_cust_data['loc_og_mou_6']=hv_cust_data['loc_og_t2f_mou_6']+hv_cust_data['loc_og_t2m_mou_6']+hv_cust_data['loc_og_t2t_mou_6']+hv_cust_data['loc_og_t2c_mou_6']
hv_cust_data['loc_og_mou_7']=hv_cust_data['loc_og_t2f_mou_7']+hv_cust_data['loc_og_t2m_mou_7']+hv_cust_data['loc_og_t2t_mou_7']+hv_cust_data['loc_og_t2c_mou_7']
hv_cust_data['loc_og_mou_active']=hv_cust_data['loc_og_t2f_mou_active']+hv_cust_data['loc_og_t2m_mou_active']+hv_cust_data['loc_og_t2t_mou_active']+hv_cust_data['loc_og_t2c_mou_active']
hv_cust_data['loc_og_mou_good']=(hv_cust_data['loc_og_mou_6']+hv_cust_data['loc_og_mou_7'])/2

In [ ]:
hv_cust_data[cols].head()


In [ ]:
print(hv_cust_data[~(hv_cust_data['loc_og_mou_good'].isnull())].groupby(['churn'])['mobile_number'].count())
print(hv_cust_data[hv_cust_data['loc_og_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())


In [ ]:
hv_cust_data['loc_og_null_good']=np.where((hv_cust_data['loc_og_mou_good'].isnull()),1,0) 
hv_cust_data['loc_og_null_active']=np.where((hv_cust_data['loc_og_mou_active'].isnull()),1,0) 
print('Good Month Comparision \n')
print(hv_cust_data[hv_cust_data['loc_og_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print(hv_cust_data[hv_cust_data['loc_og_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('Active Month Comparision \n')
print(hv_cust_data[hv_cust_data['loc_og_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())
print(hv_cust_data[hv_cust_data['loc_og_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_cust_data['loc_og_mou_active']=hv_cust_data[['loc_og_mou_active']].fillna(0)
hv_cust_data['loc_og_mou_good']=hv_cust_data[['loc_og_mou_good']].fillna(0)

print(hv_cust_data[['loc_og_mou_active','loc_og_mou_good']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
hv_custdata = hv_cust_data

In [ ]:
hv_custdata[hv_custdata['loc_og_mou_active']>np.percentile(hv_custdata['loc_og_mou_active'],99.95)][['loc_og_mou_active','loc_og_mou_good','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['loc_og_mou_active']>np.percentile(hv_custdata['loc_og_mou_active'],99.95))]

In [ ]:
hv_custdata.drop(columns=['loc_og_mou_6','loc_og_t2f_mou_6','loc_og_t2m_mou_6','loc_og_t2t_mou_6','loc_og_t2c_mou_6','loc_og_mou_7','loc_og_t2f_mou_7','loc_og_t2m_mou_7','loc_og_t2t_mou_7','loc_og_t2c_mou_7',
                          'loc_og_t2f_mou_active','loc_og_t2m_mou_active','loc_og_t2t_mou_active','loc_og_t2c_mou_active'],axis=1,inplace=True)

In [ ]:
# Feature: max_rech_data and max_rech_amt
cols=list(hv_custdata.filter(regex='max_rech').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['max_rech_amt_good']=(hv_custdata['max_rech_amt_6']+hv_custdata['max_rech_amt_7'])/2
hv_custdata[['max_rech_amt_good','max_rech_amt_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['max_rech_amt_active']>np.percentile(hv_custdata['max_rech_amt_active'],99.97)][['max_rech_amt_good','max_rech_amt_active','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['max_rech_amt_active']>np.percentile(hv_custdata['max_rech_amt_active'],99.97))]
hv_custdata['max_rech_data_good']=(hv_custdata['max_rech_data_6']+hv_custdata['max_rech_data_7'])/2
print('Good Month Distribution \n ')
print(hv_custdata[hv_custdata['max_rech_data_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\nActive Month Distribution \n ')
print(hv_custdata[hv_custdata['max_rech_data_active'].isnull()].groupby(['churn'])['mobile_number'].count())
hv_custdata[hv_custdata['max_rech_data_active']==0].head()

In [ ]:
hv_custdata['max_rech_data_null_good']=np.where((hv_custdata['max_rech_data_good'].isnull()),1,0) 
hv_custdata['max_rech_data_null_active']=np.where((hv_custdata['max_rech_data_active'].isnull()),1,0) 
hv_custdata['max_rech_data_good']=hv_custdata['max_rech_data_good'].fillna(0)
hv_custdata['max_rech_data_active']=hv_custdata['max_rech_data_active'].fillna(0)
hv_custdata.drop(columns=['max_rech_data_6','max_rech_data_7','max_rech_amt_6','max_rech_amt_7'],axis=1,inplace=True)

In [ ]:
# Feature nich pack user
cols=list(hv_custdata.filter(regex='night_pck').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['night_pck_user_good']=(hv_custdata['night_pck_user_6']+hv_custdata['night_pck_user_7'])/2
hv_custdata['night_pck_user_null_good']=np.where((hv_custdata['night_pck_user_good'].isnull()),1,0) 
hv_custdata['night_pck_user_null_active']=np.where((hv_custdata['night_pck_user_active'].isnull()),1,0) 

print('Good Month Distribution \n ')
print(hv_custdata[hv_custdata['night_pck_user_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\nActive Month Distribution \n ')
print(hv_custdata[hv_custdata['night_pck_user_active'].isnull()].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['night_pck_user_good']=hv_custdata['night_pck_user_good'].fillna(0)
hv_custdata['night_pck_user_active']=hv_custdata['night_pck_user_active'].fillna(0)
hv_custdata[['night_pck_user_good','night_pck_user_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata.drop(columns=['night_pck_user_6','night_pck_user_7'],axis=1,inplace=True)

In [ ]:
# Feature : offnet and onnet users
cols=list(hv_custdata.filter(regex='.net_mou').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['onnet_mou_good']=(hv_custdata['onnet_mou_6']+hv_custdata['onnet_mou_7'])/2
hv_custdata['offnet_mou_good']=(hv_custdata['offnet_mou_6']+hv_custdata['offnet_mou_7'])/2

hv_custdata['onnet_mou_null_good']=np.where((hv_custdata['onnet_mou_good'].isnull()),1,0) 
hv_custdata['onnet_mou_null_active']=np.where((hv_custdata['onnet_mou_active'].isnull()),1,0) 

hv_custdata['offnet_mou_null_good']=np.where((hv_custdata['offnet_mou_good'].isnull()),1,0) 
hv_custdata['offnet_mou_null_active']=np.where((hv_custdata['offnet_mou_active'].isnull()),1,0) 

print('Onnet Good Month Distribution \n ')
print(hv_custdata[hv_custdata['onnet_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Onnet Active Month Distribution \n ')
print(hv_custdata[hv_custdata['onnet_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('Offnet Good Month Distribution \n ')
print(hv_custdata[hv_custdata['offnet_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Offent Active Month Distribution \n ')
print(hv_custdata[hv_custdata['offnet_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['onnet_mou_good']=hv_custdata['onnet_mou_good'].fillna(0)
hv_custdata['onnet_mou_active']=hv_custdata['onnet_mou_active'].fillna(0)
hv_custdata['offnet_mou_good']=hv_custdata['offnet_mou_good'].fillna(0)
hv_custdata['offnet_mou_active']=hv_custdata['offnet_mou_active'].fillna(0)

print(hv_custdata[['onnet_mou_good','onnet_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))
print(hv_custdata[['offnet_mou_good','offnet_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00]))

In [ ]:
print(hv_custdata[hv_custdata['onnet_mou_active']>np.percentile(hv_custdata['onnet_mou_active'],99.95)][['onnet_mou_good','onnet_mou_active','churn']])
print(hv_custdata[hv_custdata['offnet_mou_active']>np.percentile(hv_custdata['offnet_mou_active'],99.95)][['offnet_mou_good','offnet_mou_active','churn']])

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['onnet_mou_active']>np.percentile(hv_custdata['onnet_mou_active'],99.95))]
hv_custdata=hv_custdata[~(hv_custdata['offnet_mou_active']>np.percentile(hv_custdata['offnet_mou_active'],99.95))]
hv_custdata.drop(columns=['offnet_mou_6','offnet_mou_7','onnet_mou_6','onnet_mou_7'],axis=1,inplace=True)

In [ ]:
# Feature : sanchet 2g and 3g
cols=list(hv_custdata.filter(regex='sachet_').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['sachet_2g_good']=(hv_custdata['sachet_2g_6']+hv_custdata['sachet_2g_7'])/2
hv_custdata['sachet_3g_good']=(hv_custdata['sachet_3g_6']+hv_custdata['sachet_3g_7'])/2
hv_custdata[['sachet_2g_good','sachet_3g_good','sachet_2g_active','sachet_2g_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['sachet_2g_good']>np.percentile(hv_custdata['sachet_2g_good'],99.99)][['sachet_2g_good','sachet_3g_good','sachet_2g_active','sachet_3g_active','churn']]

In [ ]:

hv_custdata.drop(columns=['sachet_2g_6','sachet_2g_7','sachet_3g_6','sachet_3g_7'],axis=1,inplace=True)

In [ ]:
# Feature: std incoming calls
cols=list(hv_custdata.filter(regex='std_ic').columns)
hv_custdata[cols].head()

In [ ]:
# It is evident that the column std_ic_mou_% is aggregated column of other variables
hv_custdata['std_ic_mou_6']=hv_custdata['std_ic_t2f_mou_6']+hv_custdata['std_ic_t2m_mou_6']+hv_custdata['std_ic_t2t_mou_6']
hv_custdata['std_ic_mou_7']=hv_custdata['std_ic_t2f_mou_6']+hv_custdata['std_ic_t2m_mou_6']+hv_custdata['std_ic_t2t_mou_6']
hv_custdata['std_ic_mou_active']=hv_custdata['std_ic_t2f_mou_active']+hv_custdata['std_ic_t2m_mou_active']+hv_custdata['std_ic_t2t_mou_active']
hv_custdata['std_ic_mou_good']=(hv_custdata['std_ic_mou_6']+hv_custdata['std_ic_mou_7'])/2

hv_custdata['std_ic_mou__null_good']=np.where((hv_custdata['std_ic_mou_good'].isnull()),1,0) 
hv_custdata['std_ic_mou_null_active']=np.where((hv_custdata['std_ic_mou_active'].isnull()),1,0) 

print('STD IC NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['std_ic_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n STD IC NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['std_ic_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('STD IC 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['std_ic_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n STD IC 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['std_ic_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['std_ic_mou_good']=hv_custdata['std_ic_mou_good'].fillna(0)
hv_custdata['std_ic_mou_active']=hv_custdata['std_ic_mou_active'].fillna(0)
hv_custdata[['std_ic_mou_good','std_ic_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['std_ic_mou_active']>np.percentile(hv_custdata['std_ic_mou_active'],99.95)][['std_ic_mou_active','std_ic_mou_good','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['std_ic_mou_active']>np.percentile(hv_custdata['std_ic_mou_active'],99.95))]
hv_custdata.drop(columns=['std_ic_mou_6','std_ic_mou_7','std_ic_t2f_mou_6','std_ic_t2f_mou_7','std_ic_t2f_mou_active','std_ic_t2m_mou_6',
                 'std_ic_t2m_mou_7','std_ic_t2m_mou_active','std_ic_t2t_mou_6','std_ic_t2t_mou_7','std_ic_t2t_mou_active'],axis=1,inplace=True)

In [ ]:
cols=list(hv_custdata.filter(regex='roam_ic').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['roam_ic_mou_good']=(hv_custdata['roam_ic_mou_6']+hv_custdata['roam_ic_mou_7'])/2

hv_custdata['roam_ic_mou_null_good']=np.where((hv_custdata['roam_ic_mou_good'].isnull()),1,0) 
hv_custdata['roam_ic_mou_null_active']=np.where((hv_custdata['roam_ic_mou_active'].isnull()),1,0) 

print('Roam IC NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_ic_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Roam IC NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_ic_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('Roam IC 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_ic_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n Roam IC 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_ic_mou_active']==0].groupby(['churn'])['mobile_number'].count())


In [ ]:
hv_custdata['roam_ic_mou_good']=hv_custdata['roam_ic_mou_good'].fillna(0)
hv_custdata['roam_ic_mou_active']=hv_custdata['roam_ic_mou_active'].fillna(0)
hv_custdata[['roam_ic_mou_good','roam_ic_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['roam_ic_mou_active']>np.percentile(hv_custdata['roam_ic_mou_active'],99.99)][['roam_ic_mou_good','roam_ic_mou_active','churn']]

In [ ]:
hv_custdata.drop(columns=['roam_ic_mou_6','roam_ic_mou_7'],axis=1,inplace=True)

In [ ]:
# Feature: special incoming
cols=list(hv_custdata.filter(regex='spl_ic').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['spl_ic_mou_good']=(hv_custdata['spl_ic_mou_6']+hv_custdata['spl_ic_mou_7'])/2

hv_custdata['spl_ic_mou_null_good']=np.where((hv_custdata['spl_ic_mou_good'].isnull()),1,0) 
hv_custdata['spl_ic_mou_null_active']=np.where((hv_custdata['spl_ic_mou_active'].isnull()),1,0) 

print('Special IC NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_ic_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Special IC NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_ic_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('Special IC 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_ic_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n Special IC 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_ic_mou_active']==0].groupby(['churn'])['mobile_number'].count())


In [ ]:
hv_custdata['spl_ic_mou_good']=hv_custdata['spl_ic_mou_good'].fillna(0)
hv_custdata['spl_ic_mou_active']=hv_custdata['spl_ic_mou_active'].fillna(0)
hv_custdata[['spl_ic_mou_good','spl_ic_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['spl_ic_mou_good']>np.percentile(hv_custdata['spl_ic_mou_good'],99.98)][['spl_ic_mou_good','spl_ic_mou_active','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['spl_ic_mou_good']>np.percentile(hv_custdata['spl_ic_mou_good'],99.98))]
hv_custdata.drop(columns=['spl_ic_mou_6','spl_ic_mou_7'],axis=1,inplace=True)

In [ ]:
# Feature : Other ic
cols=list(hv_custdata.filter(regex='ic_others').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['ic_others_good']=(hv_custdata['ic_others_6']+hv_custdata['ic_others_7'])/2

hv_custdata['ic_others_null_good']=np.where((hv_custdata['ic_others_good'].isnull()),1,0) 
hv_custdata['ic_others_null_active']=np.where((hv_custdata['ic_others_active'].isnull()),1,0) 

print('Other IC NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['ic_others_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Other IC NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['ic_others_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('Other IC 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['ic_others_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n Other IC 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['ic_others_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['ic_others_good']=hv_custdata['ic_others_good'].fillna(0)
hv_custdata['ic_others_active']=hv_custdata['ic_others_active'].fillna(0)
hv_custdata[['ic_others_good','ic_others_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['ic_others_good']>np.percentile(hv_custdata['ic_others_good'],99.97)][['ic_others_good','ic_others_active','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['ic_others_good']>np.percentile(hv_custdata['ic_others_good'],99.97))]
hv_custdata.drop(columns=['ic_others_6','ic_others_7'],axis=1,inplace=True)

In [ ]:
# Feature : isd ic
cols=list(hv_custdata.filter(regex='isd_ic').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['isd_ic_mou_good']=(hv_custdata['isd_ic_mou_6']+hv_custdata['isd_ic_mou_7'])/2

hv_custdata['isd_ic_mou_null_good']=np.where((hv_custdata['isd_ic_mou_good'].isnull()),1,0) 
hv_custdata['isd_ic_mou_null_active']=np.where((hv_custdata['isd_ic_mou_active'].isnull()),1,0) 

print('ISD IC NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_ic_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n ISD IC NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_ic_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('ISD IC 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_ic_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n ISD ic 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_ic_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['isd_ic_mou_good']=hv_custdata['isd_ic_mou_good'].fillna(0)
hv_custdata['isd_ic_mou_active']=hv_custdata['isd_ic_mou_active'].fillna(0)
hv_custdata[['isd_ic_mou_good','isd_ic_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['isd_ic_mou_good']>np.percentile(hv_custdata['isd_ic_mou_good'],99.95)][['isd_ic_mou_good','isd_ic_mou_active','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['isd_ic_mou_good']>np.percentile(hv_custdata['isd_ic_mou_good'],99.95))]
hv_custdata.drop(columns=['isd_ic_mou_6','isd_ic_mou_7'],axis=1,inplace=True)

In [ ]:
# Feature: total incoming
cols=list(hv_custdata.filter(regex='total_ic_mou').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['total_ic_mou_good']=(hv_custdata['total_ic_mou_6']+hv_custdata['total_ic_mou_7'])/2

In [ ]:
hv_custdata[['total_ic_mou_good','std_ic_mou_good','roam_ic_mou_good','spl_ic_mou_good','loc_ic_mou_good']].head()

In [ ]:
# total incoming feature is the aggregared feature
hv_custdata.drop(columns=['total_ic_mou_6','total_ic_mou_7','total_ic_mou_good','total_ic_mou_active'],axis=1,inplace=True)

In [ ]:
# Feature out going calls
cols=list(hv_custdata.filter(regex='std_og').columns)
hv_custdata[cols].head()


In [ ]:
hv_custdata['std_og_mou_6']=hv_custdata['std_og_t2f_mou_6']+hv_custdata['std_og_t2m_mou_6']+hv_custdata['std_og_t2t_mou_6']
hv_custdata['std_og_mou_7']=hv_custdata['std_og_t2f_mou_6']+hv_custdata['std_og_t2m_mou_6']+hv_custdata['std_og_t2t_mou_6']
hv_custdata['std_og_mou_active']=hv_custdata['std_og_t2f_mou_active']+hv_custdata['std_og_t2m_mou_active']+hv_custdata['std_og_t2t_mou_active']
hv_custdata['std_og_mou_good']=(hv_custdata['std_og_mou_6']+hv_custdata['std_og_mou_7'])/2

hv_custdata['std_og_mou__null_good']=np.where((hv_custdata['std_og_mou_good'].isnull()),1,0) 
hv_custdata['std_og_mou_null_active']=np.where((hv_custdata['std_og_mou_active'].isnull()),1,0) 

print('STD OG NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['std_og_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n STD OG NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['std_og_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('STD OG 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['std_og_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n STD OG 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['std_og_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['std_og_mou_good']=hv_custdata['std_og_mou_good'].fillna(0)
hv_custdata['std_og_mou_active']=hv_custdata['std_og_mou_active'].fillna(0)
hv_custdata[['std_og_mou_good','std_og_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['std_og_mou_good']>np.percentile(hv_custdata['std_og_mou_good'],99.99)][['std_og_mou_good','std_og_mou_active','churn']]

In [ ]:
hv_custdata.drop(columns=['std_og_mou_6','std_og_mou_7','std_og_t2f_mou_6','std_og_t2f_mou_7','std_og_t2f_mou_active','std_og_t2m_mou_6',
                 'std_og_t2m_mou_7','std_og_t2m_mou_active','std_og_t2t_mou_6','std_og_t2t_mou_7','std_og_t2t_mou_active'],axis=1,inplace=True)

In [ ]:
cols=list(hv_custdata.filter(regex='roam_og').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['roam_og_mou_good']=(hv_custdata['roam_og_mou_6']+hv_custdata['roam_og_mou_7'])/2

hv_custdata['roam_og_mou_null_good']=np.where((hv_custdata['roam_og_mou_good'].isnull()),1,0) 
hv_custdata['roam_og_mou_null_active']=np.where((hv_custdata['roam_og_mou_active'].isnull()),1,0) 

print('Roam OG NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_og_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Roam OG NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_og_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('Roam OG 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_og_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n Roam OG 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['roam_og_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['roam_og_mou_good']=hv_custdata['roam_og_mou_good'].fillna(0)
hv_custdata['roam_og_mou_active']=hv_custdata['roam_og_mou_active'].fillna(0)
hv_custdata[['roam_og_mou_good','roam_og_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['roam_og_mou_active']>np.percentile(hv_custdata['roam_og_mou_active'],99.99)][['roam_og_mou_good','roam_og_mou_active','churn']]

In [ ]:
hv_custdata.drop(columns=['roam_og_mou_6','roam_og_mou_7'],axis=1,inplace=True)

In [ ]:
# Feature: special outgoing
cols=list(hv_custdata.filter(regex='spl_og').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['spl_og_mou_good']=(hv_custdata['spl_og_mou_6']+hv_custdata['spl_og_mou_7'])/2

hv_custdata['spl_og_mou_null_good']=np.where((hv_custdata['spl_og_mou_good'].isnull()),1,0) 
hv_custdata['spl_og_mou_null_active']=np.where((hv_custdata['spl_og_mou_active'].isnull()),1,0) 

print('Special OG NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_og_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Special OG NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_og_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('Special OG 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_og_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n Special OG 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['spl_og_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['spl_og_mou_good']=hv_custdata['spl_og_mou_good'].fillna(0)
hv_custdata['spl_og_mou_active']=hv_custdata['spl_og_mou_active'].fillna(0)
hv_custdata[['spl_og_mou_good','spl_og_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['spl_og_mou_active']>np.percentile(hv_custdata['spl_og_mou_active'],99.96)][['spl_og_mou_good','spl_og_mou_active','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['spl_og_mou_active']>np.percentile(hv_custdata['spl_og_mou_active'],99.96))]
hv_custdata.drop(columns=['spl_og_mou_6','spl_og_mou_7'],axis=1,inplace=True)

In [ ]:
cols=list(hv_custdata.filter(regex='og_others').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['og_others_good']=(hv_custdata['og_others_6']+hv_custdata['og_others_7'])/2

hv_custdata['og_others_null_good']=np.where((hv_custdata['og_others_good'].isnull()),1,0) 
hv_custdata['og_others_null_active']=np.where((hv_custdata['og_others_active'].isnull()),1,0) 

print('Other OG NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['og_others_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n Other OG NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['og_others_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('Other OG 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['og_others_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n Other OG 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['og_others_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['og_others_good']=hv_custdata['og_others_good'].fillna(0)
hv_custdata['og_others_active']=hv_custdata['og_others_active'].fillna(0)
hv_custdata[['og_others_good','og_others_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata[hv_custdata['og_others_good']>np.percentile(hv_custdata['og_others_good'],99.995)][['og_others_good','og_others_active','churn']]

In [ ]:
hv_custdata.drop(columns=['og_others_6','og_others_7'],axis=1,inplace=True)

# Feature: isd outgoing
cols=list(hv_custdata.filter(regex='isd_og').columns)
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['isd_og_mou_good']=(hv_custdata['isd_og_mou_6']+hv_custdata['isd_og_mou_7'])/2

hv_custdata['isd_og_mou_null_good']=np.where((hv_custdata['isd_og_mou_good'].isnull()),1,0) 
hv_custdata['isd_og_mou_null_active']=np.where((hv_custdata['isd_og_mou_active'].isnull()),1,0) 

print('ISD OG NaN Good Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_og_mou_good'].isnull()].groupby(['churn'])['mobile_number'].count())
print('\n ISD OG NaN Active Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_og_mou_active'].isnull()].groupby(['churn'])['mobile_number'].count())

print('ISD OG 0 Good Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_og_mou_good']==0].groupby(['churn'])['mobile_number'].count())
print('\n isd OG 0 Active Month Distribution \n ')
print(hv_custdata[hv_custdata['isd_og_mou_active']==0].groupby(['churn'])['mobile_number'].count())

In [ ]:
hv_custdata['isd_og_mou_good']=hv_custdata['isd_og_mou_good'].fillna(0)
hv_custdata['isd_og_mou_active']=hv_custdata['isd_og_mou_active'].fillna(0)
hv_custdata[['isd_og_mou_good','isd_og_mou_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])


In [ ]:
hv_custdata[hv_custdata['isd_og_mou_active']>np.percentile(hv_custdata['isd_og_mou_active'],99.99)][['isd_og_mou_active','isd_og_mou_good','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['isd_og_mou_active']>np.percentile(hv_custdata['isd_og_mou_active'],99.99))]
hv_custdata.drop(columns=['isd_og_mou_6','isd_og_mou_7'],axis=1,inplace=True)
# Drop total outgoing features which are aggregated features
hv_custdata.drop(columns=['total_og_mou_6','total_og_mou_7','total_og_mou_active'],axis=1,inplace=True)
hv_custdata.drop(columns=['total_data_rech_amt_6','total_data_rech_amt_7','total_rech_data_6','total_rech_data_7','total_rech_data_active'],axis=1,inplace=True)

In [ ]:
hv_custdata['av_rech_amt_data_good']=(hv_custdata['av_rech_amt_data_6']+hv_custdata['av_rech_amt_data_7'])/2
hv_custdata[['av_rech_amt_data_good','av_rech_amt_data_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])


In [ ]:
hv_custdata[hv_custdata['av_rech_amt_data_good']>np.percentile(hv_custdata['av_rech_amt_data_good'],99.99)][['av_rech_amt_data_good','av_rech_amt_data_active','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['av_rech_amt_data_good']>np.percentile(hv_custdata['av_rech_amt_data_good'],99.99))]

hv_custdata.drop(columns=['av_rech_amt_data_6','av_rech_amt_data_7'],axis=1,inplace=True)

cols=['total_rech_num_6','total_rech_num_7','total_rech_num_active','total_rech_amt_6','total_rech_amt_7','total_rech_amt_active']
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['avg_rech_amt_good']=(hv_custdata['total_rech_amt_6']+hv_custdata['total_rech_amt_7'])/(hv_custdata['total_rech_num_6']+hv_custdata['total_rech_num_7'])
hv_custdata['avg_rech_amt_active']=hv_custdata['total_rech_amt_active']/hv_custdata['total_rech_num_active']

In [ ]:
hv_custdata[['avg_rech_amt_good','avg_rech_amt_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata['avg_rech_amt_null_active']=np.where((hv_custdata['avg_rech_amt_active'].isnull()),1,0) 
hv_custdata['avg_rech_amt_active']=hv_custdata['avg_rech_amt_active'].fillna(0)

In [ ]:
hv_custdata[hv_custdata['avg_rech_amt_good']>np.percentile(hv_custdata['avg_rech_amt_good'],99.99)][['avg_rech_amt_good','avg_rech_amt_active','churn']]

In [ ]:
hv_custdata=hv_custdata[~(hv_custdata['avg_rech_amt_good']>np.percentile(hv_custdata['avg_rech_amt_good'],99.99))]

hv_custdata.drop(columns=cols,axis=1,inplace=True)

cols=['last_day_rch_amt_6','last_day_rch_amt_7','last_day_rch_amt_active']
print(hv_custdata[cols].isnull().sum(axis=0).sort_values(ascending=False))

In [ ]:
hv_custdata['last_day_rch_amt_good']=(hv_custdata['last_day_rch_amt_6']+hv_custdata['last_day_rch_amt_7'])/2
hv_custdata[['last_day_rch_amt_good','last_day_rch_amt_active']].describe(percentiles=[0.75,.995,.998,.999,1.00])

In [ ]:
hv_custdata.drop(columns=['last_day_rch_amt_6','last_day_rch_amt_7'],axis=1,inplace=True)

hv_custdata['churn_text']=hv_custdata['churn'].apply(lambda x: 'churn' if x==1 else 'no-churn')

In [ ]:
# Derive new features using the dates
hv_custdata['rech_before_lastday_6'] = (hv_custdata['last_date_of_month_6']-hv_custdata['date_of_last_rech_6']).apply(lambda x: x.days)
hv_custdata['rech_before_lastday_7'] = (hv_custdata['last_date_of_month_7']-hv_custdata['date_of_last_rech_7']).apply(lambda x: x.days)
hv_custdata['rech_before_lastday_active'] = (hv_custdata['last_date_of_month_active']-hv_custdata['date_of_last_rech_active']).apply(lambda x: x.days)

hv_custdata['rechdata_before_lastday_6'] = (hv_custdata['last_date_of_month_6']-hv_custdata['date_of_last_rech_data_6']).apply(lambda x: x.days)
hv_custdata['rechdata_before_lastday_7'] = (hv_custdata['last_date_of_month_7']-hv_custdata['date_of_last_rech_data_7']).apply(lambda x: x.days)
hv_custdata['rechdata_before_lastday_active'] = (hv_custdata['last_date_of_month_active']-hv_custdata['date_of_last_rech_data_active']).apply(lambda x: x.days)

In [ ]:
hv_custdata['rech_before_lastday_good']=(hv_custdata['rech_before_lastday_6']+hv_custdata['rech_before_lastday_7'])/2
hv_custdata['rechdata_before_lastday_good']=(hv_custdata['rechdata_before_lastday_6']+hv_custdata['rechdata_before_lastday_7'])/2

In [ ]:
hv_custdata.drop(columns=['rechdata_before_lastday_6','rechdata_before_lastday_7','rech_before_lastday_6','rech_before_lastday_7','date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_active','date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_active'],axis=1,inplace=True)

In [ ]:
hv_custdata.head()

In [ ]:
#Drop columns that have same values
hv_custdata_unique=hv_custdata.nunique().sort_values()
unique_cols=hv_custdata_unique[hv_custdata_unique==1].index
hv_custdata.drop(columns=unique_cols,axis=1,inplace=True)

# EDA - Data Visualisation and Inferences

In [ ]:
# Define function to plot averages of churn and no-churn cases
def avg_bar_plot(cols):
    
    fig=plt.figure(figsize=(12, 14), dpi=70, facecolor='w', edgecolor='k')
    k=1
    for i in cols:
        plt.subplot(3,3,k)
        sub_cols=list(i)+['churn']
        a=hv_custdata[sub_cols].groupby('churn')[list(i)].mean().reset_index()
        d=pd.melt(a, id_vars=['churn'], value_vars=list(i))
        sns.barplot(x="variable", y="value", hue="churn", data=d)
        plt.xticks()
        k=k+1
        
    plt.show()
    
# Define function to plot moving averages
def mov_avg_plot(i,j):
    temp=hv_custdata[['churn',i,j]]
    ax=temp[temp['churn']==0][j].rolling(window=1000).mean().plot()
    ax=temp[temp['churn']==0][i].rolling(window=1000).mean().plot()
    ax=temp[temp['churn']==1][j].rolling(window=1000).mean().plot()
    ax=temp[temp['churn']==1][i].rolling(window=1000).mean().plot()
    ax.legend([j+ ' No-Churn',i+ ' No-Churn',j+ ' Churn',i+ ' Churn'])
    plt.show()
    
    
# Define function for a stacked plot
def stacked_plot(cols):
    k=1
    for w in cols:
        x_value=w[0]
        labl=w[1]
        hv_custdata[x_value]=hv_custdata[x_value].apply(lambda x:'y' if x==1 else 'n')
        hv_custdata_totals=hv_custdata.groupby(by=[x_value,'churn_text'])['mobile_number'].count().reset_index()
        hv_custdata_analysis_totals=pd.pivot_table(hv_custdata_totals,values='mobile_number',index=x_value,columns='churn_text').reset_index()
        #print(hv_custdata_analysis_totals)
        r = range(len(hv_custdata_analysis_totals[x_value]))
    
        cust_total = [i+j for i,j in zip(hv_custdata_analysis_totals['churn'],hv_custdata_analysis_totals['no-churn'])]
        cust_nochurn = [i / j * 100 for i,j in zip(hv_custdata_analysis_totals['no-churn'], cust_total)]
        cust_churn = [i / j * 100 for i,j in zip(hv_custdata_analysis_totals['churn'], cust_total)]
     
        df_percents=pd.DataFrame({x_value:hv_custdata_analysis_totals[x_value],'Churned_off_perc':cust_churn,'No_churn_perc':cust_nochurn})
        #print(df_percents)
        sns.set(style="ticks")
        col_mat=['b','r','g','y','c','m']
        m=np.random.randint(1,5)
    
        ax=df_percents.plot.bar(stacked=True,color=[col_mat[m],col_mat[m-1]])
        #print(ax.patches)
        for p in ax.patches:
            width, height = p.get_width(), p.get_height()
            x, y = p.get_xy() 
            if height!=0:
                ax.annotate('{:.2f} %'.format(height), (p.get_x()+.25*width, p.get_y()+.5*height),
                        ha='center',
                        va='center',
                        rotation='horizontal',
                        size='smaller')   
       
        plt.xticks(r,hv_custdata_analysis_totals[x_value],rotation='horizontal')
        plt.xlabel(labl)
        plt.legend(loc='best', bbox_to_anchor=(1,1), ncol=1)
        hv_custdata[x_value]=hv_custdata[x_value].apply(lambda x:1 if x=='y' else 0)
        k=k+1
        plt.show()

        
#Define a function to study the trend of service usage across the months
def usage_across_time(p):
    k=1
    fig=plt.figure(figsize=(12, 14), dpi=70, facecolor='w', edgecolor='k')
    for cols in p:
        x=cols+'_6'
        y=cols+'_7'
        z=cols+'_active'
        b0=hv_custdata_copy.groupby(['last_date_of_month_6','churn'])[x].mean().reset_index()
        b1=hv_custdata_copy.groupby(['last_date_of_month_7','churn'])[y].mean().reset_index()
        b2=hv_custdata_copy.groupby(['last_date_of_month_active','churn'])[z].mean().reset_index()
        b0.rename({'last_date_of_month_6': 'last_date_of_month',x:cols}, axis='columns',inplace=True)
        b1.rename({'last_date_of_month_7': 'last_date_of_month',y:cols}, axis='columns',inplace=True)
        b2.rename({'last_date_of_month_active': 'last_date_of_month',z:cols}, axis='columns',inplace=True)
        b0=b0.append(b1)
        c=b0.append(b2)
        #print(c)
        plt.subplot(3,2,k)
        sns.pointplot(x='last_date_of_month',y=cols,hue='churn',data=c)
        plt.xticks(rotation='vertical')
        k=k+1
        
    plt.show()

In [ ]:
mov_avg_plot('arpu_good','arpu_active')

In [ ]:
avg_bar_plot([('spl_ic_mou_good','spl_ic_mou_active'),('ic_others_good','ic_others_active'),
              ('isd_ic_mou_good','isd_ic_mou_active'),('loc_ic_mou_good','loc_ic_mou_active'),
              ('roam_ic_mou_good','roam_ic_mou_active'),('std_ic_mou_good','std_ic_mou_active')])

In [ ]:
avg_bar_plot([('spl_og_mou_good','spl_og_mou_active')])

In [ ]:
avg_bar_plot([('spl_og_mou_good','spl_og_mou_active'),
              ('og_others_good','og_others_active'),
              ('isd_og_mou_good','isd_og_mou_active'),
              ('loc_og_mou_good','loc_og_mou_active'),
              ('roam_og_mou_good','roam_og_mou_active'),
              ('std_og_mou_good','std_og_mou_active')])

In [ ]:
avg_bar_plot([('vol_2g_mb_good','vol_2g_mb_active'),
              ('vol_3g_mb_good','vol_3g_mb_active'),
              ('vbc_3g_good','vbc_3g_good'),
              ('sachet_2g_good', 'sachet_2g_active'),
              ('monthly_2g_good', 'monthly_2g_active'),
              ('monthly_3g_good', 'monthly_3g_active')])

In [ ]:
avg_bar_plot([('onnet_mou_good','onnet_mou_active'),
              ('offnet_mou_good','offnet_mou_active'),
              ('night_pck_user_good','night_pck_user_active'),
              ('sachet_3g_good','sachet_3g_active'),
              ('count_rech_2g_good','count_rech_2g_active'),
              ('count_rech_3g_good','count_rech_3g_active')])

In [ ]:
avg_bar_plot([('arpu_good','arpu_active'),
              ('av_rech_amt_data_good','av_rech_amt_data_active'),
              ('avg_rech_amt_good','avg_rech_amt_active'),
              ('last_day_rch_amt_good','last_day_rch_amt_active'),
              ('max_rech_amt_good','max_rech_amt_active'),
              ('max_rech_data_good','max_rech_data_active')])

In [ ]:
stacked_plot([('ic_others_null_active','Active Month IC others Null Indicator'),
              ('ic_others_null_good','Good Month Month IC others Null Indicator'),
              ('isd_ic_mou_null_active','Active Month ISD IC Null Indicator'),
              ('isd_ic_mou_null_good','Good Month ISD IC Null Indicator'),
              ('loc_ic_null_active','Active Month Local IC Null Indicator'),
              ('loc_ic_null_good','Good Month Local IC Null Indicator'),
              ('roam_ic_mou_null_active','Active Month Roaming IC Null Indicator'),
              ('roam_ic_mou_null_good','Good Month Roaming IC Null Indicator'),
              ('spl_ic_mou_null_active','Active Month Special IC Null Indicator'),
              ('spl_ic_mou_null_good','Good Month Special IC Null Indicator'),
              ('std_ic_mou__null_good','Good Month STD IC Null Indicator'),
              ('std_ic_mou_null_active','Active Month STD IC Null Indicator')])

In [ ]:
stacked_plot([('og_others_null_active','Active Month OG others Null Indicator'),
              ('og_others_null_good','Good Month Month OG others Null Indicator'),
              ('isd_og_mou_null_active','Active Month ISD OG Null Indicator'),
              ('isd_og_mou_null_good','Good Month ISD OG Null Indicator'),
              ('loc_og_null_active','Active Month Local OG Null Indicator'),
              ('loc_og_null_good','Good Month Local OG Null Indicator'),
              ('roam_og_mou_null_active','Active Month Roaming OG Null Indicator'),
              ('roam_og_mou_null_good','Good Month Roaming OG Null Indicator'),
              ('spl_og_mou_null_active','Active Month Special OG Null Indicator'),
              ('spl_og_mou_null_good','Good Month Special OG Null Indicator'),
              ('std_og_mou__null_good','Good Month STD OG Null Indicator'),
              ('std_og_mou_null_active','Active Month STD OG Null Indicator')])

In [ ]:
stacked_plot([('max_rech_data_null_active','Active Month Max Rech Data Null Indicator'),
              ('max_rech_data_null_good','Good Month Max Rech Data Null Indicator'),
              ('night_pck_user_null_active','Active Month Night Pk user Null Indicator'),
              ('night_pck_user_null_good','Good Month Night Pk user Null Indicator'),
              ('offnet_mou_null_active','Active Month Offnet Null Indicator'),
              ('offnet_mou_null_active','Good Month Offnet Null Indicator'),
              ('onnet_mou_null_active','Active Month Onnet Null Indicator'),
              ('onnet_mou_null_active','Good Month Onnet Null Indicator'),
              ('avg_rech_amt_null_active','Active Month Avg Rech Amt Null Indicator')])

Findings:

From 'Moving Average plot', it is evident that,Average Revenue Per User increases drastically from active months to good months, in case of users who have churned. The increase is marginal for non-churned users - Users whose minutes of usage on incoming calls, outgoing calls reduce are more likely to churn than users whose mou patterns are unchanged.

The evidence of change in the incoming calls, indicates the customer communication contact preference and is a staunch indicator of customer using alternate service provider

Users who have reduced data volume, sachet, night-packs usage or reduced recharge count/amount are more likely to churn

Similarly users showing decreasing trend in onnet/offnet minutes of usage are more likely to churn out

# Apply Dimensionality Reduction (PCA) and Balance the class as appropriate

In [ ]:
print(hv_custdata.shape)
hv_custdata.head()

In [ ]:
#Drop columns that have same values
hv_custdata_unique=hv_custdata.nunique().sort_values()
unique_cols=hv_custdata_unique[hv_custdata_unique==1].index
hv_custdata.drop(columns=unique_cols,axis=1,inplace=True)

In [ ]:
print(hv_custdata.shape)

In [ ]:
# Standardizing the values
hv_custdata_features=hv_custdata.drop(['mobile_number','churn_text','rech_before_lastday_active','rechdata_before_lastday_active','rech_before_lastday_good','rechdata_before_lastday_good'],axis=1)
#hv_custdata_features=hv_custdata.drop(['mobile_number','churn_text'],axis=1)
cols=list(hv_custdata_features.columns)
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(hv_custdata_features)
hv_custdata_std=pd.DataFrame(scaled_df, columns=cols)

In [ ]:
# Put feature variable to x and response variable to y
x = hv_custdata_std.drop(['churn'],axis=1)
y = hv_custdata_std['churn']

# Split the set
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.7,test_size=0.3,random_state=100)
print(y_train.mean())
print(y_train.value_counts())

In [ ]:
# Create a frame and empty lists to store the predictor and actual values that help easy model visualisation
score_frame=pd.DataFrame()
algorithm_name=[]
recall_scores=[]
f1_scores=[]
accuracy_scores=[]


# Define function to predict the test data set usinf given algorithm
def run_algorithm(algo,x_train_set,y_train_set,x_test_set,y_test_set,algo_name,roc_req):
    # algo - model object
    # x_train_set - Independent variables from test set
    # y_train_set - Predictor variable from training set 
    # x_test_set - Independent variables from test set 
    # y_test_set - Predictor variable from test set
    # algo_name - Algorithm Name 
    # roc_req - Calculations of roc, acu and coefficeints
    
    algo_model = algo.fit(x_train_set,y_train_set)

    # predict values and probabilities
    y_pred=algo_model.predict(x_test_set)
    y_prob=algo_model.predict_proba(x_test_set)[:,1]
    y_pred_train=algo_model.predict(x_train_set)
    
    # Print Values
    print (algo_name +' : ')
    print('-------------------------')
    print("Classification report : ","\n", classification_report(y_test,y_pred))
    print("Accuracy   Score : ",accuracy_score(y_test,y_pred))
    print("Recall Score on train Set: ",metrics.recall_score(y_train_set,y_pred_train))
    print("Recall Score on test Set: ",metrics.recall_score(y_test,y_pred))
    
    # Set up frames for model scores for later use
    algorithm_name.append(algo_name)
    recall_scores.append(metrics.recall_score(y_test,y_pred))
    f1_scores.append(metrics.f1_score(y_test,y_pred))
    accuracy_scores.append(accuracy_score(y_test,y_pred))
    
    # Plot the ROC curve 
    if roc_req == 'y':
        model_roc_auc = roc_auc_score(y_test,y_pred) 
        print ("Area under curve : ",model_roc_auc,"\n")
        print("Confusion Matrix: ","\n",metrics.confusion_matrix(y_test,y_prob.round()))
        
        fpr,tpr,threshold = roc_curve(y_test,y_prob)
     
        plt.figure(figsize=(5, 5))
        plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % model_roc_auc)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()
    return y_test,y_prob 

In [ ]:
x_train.shape

In [ ]:
# Apply PCA on the Sampled data and Run logistic Regression
pca = PCA(svd_solver='randomized', random_state=42)
pca.fit(x_train)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
pca_final = IncrementalPCA(n_components=25)
x_train_pca = pca_final.fit_transform(x_train)
y_train_pca = y_train
x_test_pca = pca_final.transform(x_test)

print(x_test_pca.shape)
print(x_train_pca.shape)

The following strategy will be followed for class balancing -

Run the Logistic Regression to understand the current data status and accurancy model
Apply different sampling techniques - SMOTE, ADASYN and Random under sampling. For each sampling, build the model and check metrics
Select the sampling technique that gives best metrics

In [ ]:
# Run the Logistic Regression to understand the current data status
logreg = LogisticRegression(random_state=42)
dummy_var,dummy_var=run_algorithm(logreg,x_train_pca,y_train_pca,x_test_pca,y_test,'Logistic_Regression_Actual_Data','n')

In [ ]:
# Apply Adaptive Over Sampling technique
asmt = ADASYN(random_state = 42)
x_train_asmt, y_train_asmt = asmt.fit_resample(x_train_pca, y_train_pca)
print('y_train Adaptive Over Sampling Mean: ',y_train_asmt.mean(),"\n")

logreg = LogisticRegression(random_state = 42)
dummy_var,dummy_var=run_algorithm(logreg,x_train_asmt,y_train_asmt,x_test_pca,y_test,'Logistic Regression ADASYN','n')

adbc = AdaBoostClassifier(n_estimators=10,learning_rate=1)
dummy_var,dummy_var=run_algorithm(adbc,x_train_asmt,y_train_asmt,x_test_pca,y_test,'Adaboost Classifier ADASYN','n')


In [ ]:
# Apply Random Under Sampling Techique 
rusm=RandomUnderSampler(random_state = 42)
x_train_rusm, y_train_rusm = rusm.fit_resample(x_train_pca, y_train_pca)
print('y_train Random Under Sampling Mean: ',y_train_rusm.mean(),"\n")

logreg = LogisticRegression()
dummy_var,dummy_var=run_algorithm(logreg,x_train_rusm,y_train_rusm,x_test_pca,y_test,'Logistic_Regression RUS Samp','n')

print("\n")

adbc = AdaBoostClassifier(n_estimators=10,learning_rate=1)
dummy_var,dummy_var=run_algorithm(adbc,x_train_rusm,y_train_rusm,x_test_pca,y_test,'Adaboost_Classifier RUS Samp','n')



Inference :

As per the metrics above, the recall and the overall performance of the model is better when the class is balances using ADYSN Sampling technique. So, select the 'ADYSN' sampling technique for this analysis.

# Train ML models and derive the associated metrics

In [ ]:
# Apply Logistic Classifier
logreg = LogisticRegression(random_state=42)
y_test_logrec,y_prob_logrec=run_algorithm(logreg,x_train_asmt,y_train_asmt,x_test_pca,y_test,'Simple Logistic Regression','y')
score_frame['Test_Logrec']=y_test_logrec
score_frame['Prob_Logrec']=y_prob_logrec

In [ ]:
#Grid Search
logreg = LogisticRegression(random_state=42)
param = {'C':[0.001,0.003,0.005,0.01,0.03,0.05,0.1,0.3,0.5,1,2,3,3,4,5,10,20]}
clf = GridSearchCV(logreg,param,scoring='recall',refit=True,cv=10)
clf.fit(x_train_asmt,y_train_asmt)
print('Best recall: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_))

In [ ]:
# Logistic Regression with best gamma constant
logreg_cv = LogisticRegression(random_state=42, C=clf.best_params_['C'])
y_test_logrec_p,y_prob_logrec_p=run_algorithm(logreg_cv,x_train_asmt,y_train_asmt,x_test_pca,y_test,'Logistic_Reg_P','y')
score_frame['Test_Logrec_p']=y_test_logrec_p
score_frame['Prob_Logrec_p']=y_prob_logrec_p

In [ ]:
#Logistic Regression with balanced weights
logreg_bal = LogisticRegression(random_state=42, class_weight='balanced')
y_test_logrec_bw,y_prob_logrec_bw=run_algorithm(logreg_bal,x_train_pca,y_train_pca,x_test_pca,y_test,'Logistic_Regression_BW','y')
score_frame['Test_Logrec_BW']=y_test_logrec_bw
score_frame['Prob_Logrec_BW']=y_prob_logrec_bw

In [ ]:
#Logistic Regression with balanced weights and Penalty
logreg_bal_p = LogisticRegression(random_state=42, class_weight='balanced',C=clf.best_params_['C'])
y_test_logrec_bwp,y_prob_logrec_bwp=run_algorithm(logreg_bal_p,x_train_pca,y_train_pca,x_test_pca,y_test,'Logistic_Regression_BW_P','y')
score_frame['Test_Logrec_BW_P']=y_test_logrec_bwp
score_frame['Prob_Logrec_BW_P']=y_prob_logrec_bwp

In [ ]:
# Instantiate the basic random forest model
rf = RandomForestClassifier(random_state=42)
dummy_var,dummy_var=run_algorithm(rf,x_train_asmt,y_train_asmt,x_test_pca,y_test,'Random_Forest_Classifier','y')

In [ ]:
[estimator.tree_.max_depth for estimator in rf.estimators_]

In [ ]:
# GridSearchCV to find optimal maximum depth and min_samples_leaf
n_folds = 5

# parameters to build the model on
parameters = {'max_depth': range(10, 50, 4),
              'min_samples_leaf': [3,4,5,6] }

# instantiate the model
rf = RandomForestClassifier(random_state=42)

rf = GridSearchCV(rf, param_grid=parameters,
                  cv=n_folds, 
                 scoring="recall")

rf.fit(x_train_asmt, y_train_asmt)

print('\n'+'Enter the best parameters: ',rf.best_params_)

In [ ]:
# model with the best hyperparameters
rf_tuned = RandomForestClassifier(bootstrap=True,
                             max_depth=rf.best_params_['max_depth'],
                             min_samples_leaf=rf.best_params_['min_samples_leaf'],
                             n_estimators=100,
                             random_state=42)

y_test_rfc,y_prob_rfc=run_algorithm(rf_tuned,x_train_asmt,y_train_asmt,x_test_pca,y_test,'Random_Forest_Classifier_T','y')
score_frame['Test_RFC']=y_test_rfc
score_frame['Prob_RFC']=y_prob_rfc

In [ ]:
# Apply Adaboost Classifier
adbc = AdaBoostClassifier(n_estimators=50,learning_rate=1,random_state=42)
y_test_abc,y_prob_abc=run_algorithm(adbc,x_train_asmt,y_train_asmt,x_test_pca,y_test,'Adaboost_Classifier','y')
score_frame['Test_ABC']=y_test_abc
score_frame['Prob_ABC']=y_prob_abc

In [ ]:
# Apply XG Boost Regressor
xgb=XGBClassifier(random_state=42)
xgb_model = xgb.fit(x_train_asmt,y_train_asmt)

# predict values and probabilities
y_pred=xgb_model.predict(x_test_pca)

y_train_pred = xgb_model.predict(x_train_asmt)
print("Performance on train set: ", metrics.recall_score(y_train_asmt, y_train_pred.round()))

y_test_pred = xgb_model.predict(x_test_pca)
print("Performance on test set: ", metrics.recall_score(y_test, y_test_pred.round()),"\n")

# Print Values
print ('XGBooster algorithm' +' : ')
print('-------------------------')
print("Classification report : ","\n", classification_report(y_test,y_pred.round()))
print("Accuracy   Score : ",accuracy_score(y_test,y_pred.round()))
print("Recall Score on Test Set: ",metrics.recall_score(y_test,y_pred.round()))

score_frame['Test_XGBC']=y_test
score_frame['Prob_XGBC']=y_pred.round()

# Derive the prominent features and predictions

In [ ]:
# Define function for a logistic classifier and apply RFE to derive the prominent features. Check multi colienarity.

def logrec_rfe(a):
    # Feature variable = x and response variable = y
    xcols=a.columns
    
    # Use ADASYN for Over Sampling
    asmt = ADASYN(random_state = 42)
    x_train_asmt, y_train_asmt = asmt.fit_sample(a, y_train)
    print('y_train Adaptive Over Sampling Mean: ',y_train_asmt.mean(),"\n")
    
    # Set the train and test data
    x_train_samp = pd.DataFrame(data=x_train_asmt,columns=xcols)
    y_train_samp= pd.DataFrame(data=y_train_asmt,columns=['churn'])

    # Run RFE on logistic Regression
    logreg_rfe = LogisticRegression(random_state=42)
    rfe = RFE(logreg_rfe,25)             # running RFE with 25 variables as output
    rfe = rfe.fit(x_train_samp, y_train_samp)

    # Select the columns that are supported by RFE
    col = x_train_samp.columns[rfe.support_]

    x_train_sm = sm.add_constant(x_train_samp[col])
    logm2 = sm.GLM(y_train_samp,x_train_sm, family = sm.families.Binomial())
    res = logm2.fit()
    
    # Check Multi colinearity using VIF
    vif = pd.DataFrame()
    vif['features'] = x_train_samp[col].columns
    vif['VIF'] = [variance_inflation_factor(x_train_samp[col].values, i) for i in range(x_train_samp[col].shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    
    # Print stat summary and VIF table
    print(res.summary())
    print(vif)
    
    return list(vif['features'])


In [ ]:
prom_f = logrec_rfe(x_train)

In [ ]:
# Remove the variabl with Highest VIF - arpu_active
prom_f=logrec_rfe(x_train.drop(columns=['arpu_active'],axis=1))

In [ ]:
# Remove the aviable with highest p value
prom_f=logrec_rfe(x_train.drop(columns=['arpu_active','offnet_mou_active'],axis=1))

In [ ]:
# Remove the vaiables with highest VIF - 'arpu_good'
prom_f=logrec_rfe(x_train.drop(columns=['arpu_active','offnet_mou_active','arpu_good'],axis=1))

In [ ]:
# remove feature with highest VIF - 'sachet_2g_active'
prom_f=logrec_rfe(x_train.drop(columns=['arpu_active','offnet_mou_active','arpu_good','sachet_2g_active'],axis=1))

In [ ]:
# remove feature with highest VIF - 'av_rech_amt_data_active'
prom_f=logrec_rfe(x_train.drop(columns=['arpu_active','offnet_mou_active','arpu_good',
                                        'sachet_2g_active','av_rech_amt_data_active'],axis=1))

In [ ]:
avg_bar_plot([('loc_ic_mou_good','loc_ic_mou_active'),
             ('vol_3g_mb_good','vol_3g_mb_active'),
             ('sachet_3g_good','sachet_3g_active'),
             ('spl_og_mou_good','spl_og_mou_active'),
             ('std_ic_mou_good','std_ic_mou_active'),('std_og_mou_good','std_og_mou_active')])

Inference :

A selection of the most prominent features have been analysed and visualised above.
It is evident that the rate of decrease in the features - local incoming calls,special out-going calls,std calls are staunch indicators of churn behaviour.
This behaviour supports the analysis done in the exploratory stage

# Model Selection

In [ ]:
# Use the score_frame to draw consolidated ROC curve and Score matris to select the appropriate model

score_mat=pd.DataFrame(data={'Algorithm':algorithm_name,
      'Recall':recall_scores,
      'F1':f1_scores,
      'Accuracy':accuracy_scores})

print(score_mat)

cols=[('Test_Logrec','Prob_Logrec','Logistic Regression'),
      ('Test_Logrec_p','Prob_Logrec_p','Logistic Regression - Penalty Constant'),
      ('Test_Logrec_BW','Prob_Logrec_BW','Logistic Regression- Balanced Weight'),
      ('Test_RFC','Prob_RFC','Random Forest Classifier'),
      ('Test_ABC','Prob_ABC','Adaboost Classifier'),
      ('Test_XGBC','Prob_XGBC','XGB Boost Classifier')]

plt.figure(figsize=(8, 8))

for i in cols:
    fpr,tpr,threshold = roc_curve(score_frame[i[0]],score_frame[i[1]])
    plt.plot( fpr, tpr, label=i[2])


plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristics')
plt.legend(loc="lower right")

plt.show()

plt.figure(figsize=(8, 8))
for i in cols:
    precision, recall, threshold = precision_recall_curve(score_frame[i[0]],score_frame[i[1]])
    plt.plot( precision, recall, label=i[2])

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Recall vs Precision Charecteric')
plt.legend(loc="top left")
plt.show()

In [ ]:
# Rerun the selected model with Prominent features

a=x_train[prom_f]
b=x_test[prom_f]
asmt = ADASYN(random_state = 42)
x_train_asmt, y_train_asmt = asmt.fit_sample(a, y_train)
print('y_train Adaptive Over Sampling Mean: ',y_train_asmt.mean(),"\n")
    
# Set the train and test data
x_train_samp = pd.DataFrame(data=x_train_asmt,columns=prom_f)
y_train_samp= pd.DataFrame(data=y_train_asmt,columns=['churn'])

# Run Logistic Regression with Balanced Weights
y_test,y_pred=run_algorithm(logreg_bal_p,x_train_samp,y_train_samp,b,y_test,'Logistic_Regression_BW_P','y')
precision, recall, threshold = precision_recall_curve(y_test,y_pred)
plt.plot( precision, recall)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Recall vs Precision Charecteric')
plt.show()

The Top 5 prominent features to determine the customer churn behaviour are:

The sudden surge in the special incoming and out going calls in the Active months

Considerable reduction in the std incoming and out going calls

Sudden reduction in the usage of 3g data services

Sudden drop in the local incoming calls. ( The contact preferences of the customer changing)

Sudden reduction in the usage of services with validity less than a month - for example 3g Sachets
It is also evident from Exploratory Data Analysis, that where the data is not collected ( NaNs), the percentage of Churn is higher.The KPI data collection methods will need to be reviewed for a better prediction.